In [ ]:
%load_ext autoreload

%autoreload 2
%matplotlib inline

import datetime as dt

import matplotlib
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
from analysis import History, get_etfs_history, get_ticker_history

MOSCOW_TIMEZONE = pytz.timezone('Europe/Moscow')

# From rest
# client = openapi.sandbox_api_client(token)
# client.sandbox.sandbox_register_post()
# client.sandbox.sandbox_clear_post()
# client.sandbox.sandbox_currencies_balance_post(sandbox_set_currency_balance_request={"currency": "USD", "balance": 1000})


# From streaming
# candle_subs = [{'figi': 'BBG000B9XRY4', 'interval': '1min'}, {'figi': 'BBG009S39JX6', 'interval': '1min'}]
# orderbook_subs = [{'figi': 'BBG0013HGFT4', 'depth': 1}, {'figi': 'BBG009S39JX6', 'depth': 3}]
# instrument_info_subs = [{'figi': 'BBG000B9XRY4'}, {'figi': 'BBG009S39JX6'}]

# run_stream_consumer(token,
#                     candle_subs, orderbook_subs, instrument_info_subs,
#                     on_candle_event=print_event,
#                     on_orderbook_event=print_event,
#                     on_instrument_info_event=print_event)

In [ ]:
end = dt.datetime.now(dt.timezone.utc)
start = end - dt.timedelta(weeks=52)
interval = "day"
ticker = "FXGD"

# print(etfs)


# print(market.market_candles_get(figi=figi, _from=start.isoformat(), to=end.isoformat(), interval=interval))
# ['payload']['instruments'][0]['figi']

ticker_history = get_ticker_history(
    ticker=ticker, start=start, end=end, interval=interval
)

In [ ]:
plt.figure()
plt.plot(ticker_history["time"].values, ticker_history["c"])
plt.title(ticker)

In [ ]:
etfs_history, etf_tickers = get_etfs_history()

## Download and store history

In [ ]:
hist_daily = History(interval="day")
# display(hist_daily.data.time.dtype)
hist_daily.update(reload=0)
hist_daily.data

In [ ]:
hist_daily.data.time.max()

In [ ]:
data = hist_daily.data
tickers = hist_daily.tickers
# print(data['time'].dtype)

M = len(hist_daily.tickers)
cm = plt.get_cmap("gist_rainbow")
cNorm = colors.Normalize(vmin=0, vmax=M - 1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)


def get_last_not_nan(lst):
    for el in lst[::-1]:
        if not np.isnan(el):
            return el

    return np.nan


plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.set_prop_cycle(color=[scalarMap.to_rgba(i) for i in range(M)])
for ticker in tickers:
    filter = data.ticker == ticker
    t = data.loc[filter, "time"].values
    y = data.loc[filter, "c"].values  # / get_last_not_nan(data[ticker+'_c']) * 100
    #     print(ticker, y)
    plt.plot(t, y, label=ticker)

plt.ylabel("Price")
plt.xlabel("Time")
plt.legend()

plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.set_prop_cycle(color=[scalarMap.to_rgba(i) for i in range(M)])
for ticker in tickers:
    filter = data.ticker == ticker
    t = data.loc[filter, "time"].values
    y = data.loc[filter, "c"] / get_last_not_nan(data.loc[filter, "c"]) * 100
    plt.plot(t, y, label=ticker)

plt.ylabel("Price, % (100%=now)")
plt.xlabel("Time")
plt.legend()

## Basic calculate_statistics

In [ ]:
hist_daily.calculate_statistics()